# Processamento de Linguagem Natural (PLN) - Básico 

SERPRO - SUPSS - DIVISÃO DE DESENVOLVIMENTO E SUSTENTAÇÃO DE PRODUTOS COGNITIVOS

## Parte 3: Exemplo - Classificador de Texto com Deep Learning

Neste notebook iremos apresentar um exemplo de classificador de texto com Deep Learning utilizando os vários conceitos apresentados no curso até aqui.

Os dados serão sentenças categorizadas em tempo, desculpa, agradecimento, despedida e idade.

Etapas:
1. Leitura: Ler sentenças de um arquivo JSON para realizar o treinamento.
2. Pré processamento dos dados: Remover pontuação e tokenizando as sentenças
3. Criação do Vocabulário: Remover as palavras duplicadas e aplicar o Stem
4. Preparar os dados para o Treinamento: Gerar as Bag of Words e vetorização das categorias.
5. Montar a estrutura da rede neural
6. Treinamento
7. Realizar alguns testes
8. Visualização com o TensorBoard 


In [1]:
# Instalar tflearn que irá ser utilizado para criação da rede neural 
#!pip install tensorflow
#!pip install tflearn

In [2]:
# Imports
import sys
import nltk
import json
import tflearn
import random
import string
import unicodedata
import numpy as np
import tensorflow as tf
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import RSLPStemmer
from nltk.corpus import stopwords
portuguese_stops = set(stopwords.words('portuguese'))

Instructions for updating:
Colocations handled automatically by placer.


## Leitura e pré-processamento dos dados

In [3]:
# Estrutura para armazenar pontuações
tbl = dict.fromkeys(i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P'))

In [4]:
# Função para remover pontuações das sentenças
def remove_punctuation(text):
    return text.translate(tbl)

In [5]:
# Inicializa o stemmer
stemmer = SnowballStemmer(language='portuguese')
data = None

Iremos utilizar um pequeno conjunto de sentenças categorizadas para treinar modelo.

As categorias são: tempo, desculpa, agradecimento, despedida e idade

In [6]:
# Leitura do arquivo JSON com os dados de treino
with open('data/data.json') as json_data:
    data = json.load(json_data)
    print(data)

{'tempo': ['Pode informar as horas?', 'Quanto tempo passou desde que começamos?', 'isso ocorreu algum tempo atrás', 'eu falei com você semana passada', 'eu vi você ontem'], 'desculpa': ['Me desculpe', 'Ele pediu desculpas?', 'Eu não devia ter sido rude'], 'agradecimento': ['Olá, tudo bem? Muito obrigado!', 'Agradeço pela sua ajuda?', 'Muito obrigado'], 'despedida': ['Foi um prazer conhecer você', 'Adeus.', 'Vejo você em breve', 'Preciso ir agora.', 'Até logo'], 'idade': ['qual sua idade?', 'Quantos anos você tem?', 'Eu sou alguns anos mais velho que ela', 'você parece mais velho!', 'Eu tenho 30 anos. Eu sou o mais velho']}


In [7]:
# Gera uma lista com todas as categorias
categories = list(data.keys())
words = []
categories

['tempo', 'desculpa', 'agradecimento', 'despedida', 'idade']

Vamos realizar o pré processamento do texto removendo as pontuações e tokenizando as sentenças. Além disso, precisamos criar uma lista que irá conter os documentos com seu texto tokenizado e suas respectivas categorias

In [8]:
# Uma lista de tuplas com as palavras das sentenças e o nome da categoria
docs = []

for each_category in data.keys():
    for each_sentence in data[each_category]:
        # Remove a pontuação
        each_sentence = remove_punctuation(each_sentence)
        #print("Sentença:",each_sentence)
        # Extrai as palavras de cada sentença e armazena na lista
        w = nltk.word_tokenize(each_sentence)
        #print("Palavras tokenizadas:", w)
        #print('')
        words.extend(w)
        docs.append((w, each_category))

print("Exemplo de Sentença Categorizada:",docs[0])
print("Número de frases no corpus:",len(docs))
print("Corpus")
docs

Exemplo de Sentença Categorizada: (['Pode', 'informar', 'as', 'horas'], 'tempo')
Número de frases no corpus: 21
Corpus


[(['Pode', 'informar', 'as', 'horas'], 'tempo'),
 (['Quanto', 'tempo', 'passou', 'desde', 'que', 'começamos'], 'tempo'),
 (['isso', 'ocorreu', 'algum', 'tempo', 'atrás'], 'tempo'),
 (['eu', 'falei', 'com', 'você', 'semana', 'passada'], 'tempo'),
 (['eu', 'vi', 'você', 'ontem'], 'tempo'),
 (['Me', 'desculpe'], 'desculpa'),
 (['Ele', 'pediu', 'desculpas'], 'desculpa'),
 (['Eu', 'não', 'devia', 'ter', 'sido', 'rude'], 'desculpa'),
 (['Olá', 'tudo', 'bem', 'Muito', 'obrigado'], 'agradecimento'),
 (['Agradeço', 'pela', 'sua', 'ajuda'], 'agradecimento'),
 (['Muito', 'obrigado'], 'agradecimento'),
 (['Foi', 'um', 'prazer', 'conhecer', 'você'], 'despedida'),
 (['Adeus'], 'despedida'),
 (['Vejo', 'você', 'em', 'breve'], 'despedida'),
 (['Preciso', 'ir', 'agora'], 'despedida'),
 (['Até', 'logo'], 'despedida'),
 (['qual', 'sua', 'idade'], 'idade'),
 (['Quantos', 'anos', 'você', 'tem'], 'idade'),
 (['Eu', 'sou', 'alguns', 'anos', 'mais', 'velho', 'que', 'ela'], 'idade'),
 (['você', 'parece', 'mais

Agora vamos remover as palavras duplicadas e aplicar o Stem. Assim teremos o __vocabulário__ relativo ao corpus

In [9]:
# Stem de cada palavra, converte para minúsculo e remove duplicidades 
words = [stemmer.stem(w.lower()) for w in words]
words = sorted(list(set(words)))
print(words)
print('Tamanho Vocabulário:',len(words))

['30', 'adeus', 'agor', 'agradec', 'ajud', 'algum', 'alguns', 'anos', 'as', 'atrás', 'até', 'bem', 'brev', 'com', 'comec', 'conhec', 'desculp', 'desd', 'dev', 'ela', 'ele', 'em', 'eu', 'fal', 'foi', 'hor', 'idad', 'inform', 'ir', 'isso', 'log', 'mais', 'me', 'muit', 'nã', 'o', 'obrig', 'ocorr', 'olá', 'ontem', 'parec', 'pass', 'ped', 'pel', 'pod', 'praz', 'precis', 'qual', 'quant', 'que', 'rud', 'seman', 'sid', 'sou', 'sua', 'tem', 'temp', 'tenh', 'ter', 'tud', 'um', 'vej', 'velh', 'vi', 'voc']
Tamanho Vocabulário: 65


## Preparação dos dados para o Treinamento 

Vamos gerar os dados que serão utilizados no treinamento. Eles serão formados pelo Bag of Words (Matriz Binária) para cada frase e o vetor com sua respectiva categorias/labels, onde índice com valor 1 representará a categoria definida. 

In [10]:
# Cria as listas para os dados de treino
training = []
output = []

# Cria um array para o output com tamanho igual ao número de categorias
output_empty = [0] * len(categories)
print('categories:',categories)
print('output_empty:',output_empty)

for doc in docs:
    # Inicializa o bag of words para cada documento da lista
    bow = []
    # Lista de palavras tokenizadas
    token_words = doc[0]
    # Stem de cada palavra usando list comprehension
    token_words = [stemmer.stem(word.lower()) for word in token_words]
    # Cria um array com o bag of words
    for w in words:
        bow.append(1) if w in token_words else bow.append(0)

    output_row = list(output_empty)
    output_row[categories.index(doc[1])] = 1

    # Nosso conjunto de treinamento conterá um modelo bag of words e a linha de saída que informa a qual sentença pertence.
    training.append([bow, output_row])
    
print("Exemplo de dados de treino: ")# categoria 
print("Frase: ",docs[0])
print(" - BoW: ",training[0][0])
print(" - Categoria: vetor - {} , label - {} ".format(training[0][1], categories[np.argmax(training[0][1])]))

categories: ['tempo', 'desculpa', 'agradecimento', 'despedida', 'idade']
output_empty: [0, 0, 0, 0, 0]
Exemplo de dados de treino: 
Frase:  (['Pode', 'informar', 'as', 'horas'], 'tempo')
 - BoW:  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 - Categoria: vetor - [1, 0, 0, 0, 0] , label - tempo 


In [11]:
# Shuffle das nossas features e transforma em np.array enquanto o TensorFlow recebe uma matriz numérica
random.shuffle(training)
training = np.array(training)

In [12]:
# train_x contém o bag of word e train_y contém os labels/categorias
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Exemplo: train_x (BOW) -> ",train_x[0])
print("Exemplo: train_y (Labes) -> ",train_y[0])

Exemplo: train_x (BOW) ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Exemplo: train_y (Labes) ->  [0, 1, 0, 0, 0]


## Criando a da Rede Neural Profunda

Com os dados preparados vamos criar a rede neural:

- Os dados de entrada terá um shape igual ao número de features (palavras do vocabulário)
- Depois cria-se duas camadas totalmente conectadas com número de neurônios igual a 8
- A camada de saída terá uma quantidade de neurônios igual ao número de labels/categorias (5)
- Será utilizada a função de ativação softmax para gerar as probabilidades para cada categoria
- E por fim uma camada de regressão espeficando o otmizador que irá minimar a função de perda, a função de perda e a taxa de aprendizado

In [13]:
?tflearn.regression

In [14]:
# Reset do grafo tenforflow
tf.reset_default_graph()

# Cria a rede neural
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net,optimizer='adam',loss='categorical_crossentropy',learning_rate=0.001)

# Define o modelo e configura o tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

print("#Features (words)",len(train_x[0]))
print("Tamanho da camada de saída (categorias):", len(train_y[0]))

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
#Features (words) 65
Tamanho da camada de saída (categorias): 5


## Executando o Treinamento


In [15]:
# Treinamento
model.fit(train_x, train_y, n_epoch=100, batch_size=8, show_metric=True)

Training Step: 299  | total loss: 0.49909 | time: 0.003s
| Adam | epoch: 100 | loss: 0.49909 - acc: 0.9808 -- iter: 16/21
Training Step: 300  | total loss: 0.50101 | time: 0.004s
| Adam | epoch: 100 | loss: 0.50101 - acc: 0.9827 -- iter: 21/21
--


In [16]:
#Salva o modelo
model.save('models/model_dnn.tflearn')

INFO:tensorflow:/home/04449579445/workspaces/workspace_cognitiva/cursos_ia/pln_basico/models/model_dnn.tflearn is not in all_model_checkpoint_paths. Manually adding it.


## Testes

Vamos testar o modelo para algumas frases

In [17]:
# Função para ajustar os dados de teste antes de aplicar o modelo
def get_tf_record(sentence):
    global words
    # Tokenização
    sentence_words = nltk.word_tokenize(sentence)
    # Stem
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    # Bag of words
    bow = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bow[i] = 1

    return(np.array(bow))

In [18]:
sent_1 = "você falou com ele ontem?"
sent_2 = "você precisa ir agora?"
sent_3 = "gostaria de me desculpar pelas falhas no relatório!"
sent_4 = "você parece alguns anos mais velho que ela!"
sent_5 = "Estou muito agradecido pelo sua ajuda no trabalho"
sent_6 = "No passado joguei muito futebol"


In [19]:
# Previsões
print("\nImprimindo a previsão da classe das 6 sentenças de teste: ")
print(sent_1,"-> ",categories[np.argmax(model.predict([get_tf_record(sent_1)]))])
print(sent_2,"-> ",categories[np.argmax(model.predict([get_tf_record(sent_2)]))])
print(sent_3," -> ",categories[np.argmax(model.predict([get_tf_record(sent_3)]))])
print(sent_4," -> ",categories[np.argmax(model.predict([get_tf_record(sent_4)]))])
print(sent_5," -> ",categories[np.argmax(model.predict([get_tf_record(sent_5)]))])
print(sent_6," -> ",categories[np.argmax(model.predict([get_tf_record(sent_6)]))])



Imprimindo a previsão da classe das 6 sentenças de teste: 
você falou com ele ontem? ->  tempo
você precisa ir agora? ->  despedida
gostaria de me desculpar pelas falhas no relatório!  ->  desculpa
você parece alguns anos mais velho que ela!  ->  idade
Estou muito agradecido pelo sua ajuda no trabalho  ->  agradecimento
No passado joguei muito futebol  ->  desculpa


## Utilizando o Tensorboard

O tensorboard é uma ferramenta gráfica para visualizar os dados do treinamento gerados pelo TensorFlow. Por exemplos: os grafos da rede neural, métricas e dados de treinamento como imagens.

Então, vamos executar o Tensorboard para poder comparar o desempenho dos diversos treinamentos. Para executar, abra um novo terminal, ative o ambiente anaconda 'pln_basico', navegue até a pasta onde está localizado este jupyter, verifique que existe o diretório 'tflearn_logs' e execute o comando abaixo:

    tensorboard --logdir=tflearn_logs/

---

## Exercício 1

Abra o arquivo data/data.json com um editor de texto e adicione novos dados para treinamento.

Execute novamente este jupyter e veja os novos resultados

---

---

## Exercício 2

Veja que não estamos removendo stopwords neste exemplo. 

Modifique este jupyter notebook para remover stopwords tanto na preparação dos dados de treinamento quanto de teste.

Dica: a lista 'portuguese_stops' já possui as stopwords para português.
    
Execute novamente este jupyter e veja os novos resultados

---

---

## Exercício 3

Modifique o desenho da rede neural alterando a quantidade de camadas e neurônios. 

Modifique também alguns parâmetros do treinamento como número de épocas, taxa de aprendizado e batch_size. 

Faça novas execuções do jupyter e procurando alcançar um desempenho melhor do modelo. 

Dica: É recomendável fazer modificações isoladas e executar o treinamento incrementalmente até atingir a acurácia desejada.

Execute o Tensorboard para poder comparar o desempenho dos diversos treinamentos. 

---

# Fim